In [1]:
import os
import pandas as pd
import subprocess
import vcf

CLINVAR = "/dors/capra_lab/data/clinvar/2021-07-07/GRCh38/clinvar_20210626.vcf"

# Load vcf reader + 3 writers
- vcf_writer_cnc = all UTR and INTRON elements
- vcf_writer_cncomim = all UTR and INTRON elements w/ OMIM annotation
- vcf_writer_ngomim = OMIM annotation w/o UTR, INTRON, or GENEINFO

In [2]:
vcf_reader = vcf.Reader(open(CLINVAR, 'r'))

# write all non-coding
CNC = "/dors/capra_lab/data/clinvar/2021-07-07/GRCh38/clinvar_20210626.candidate_non-coding.vcf"
vcf_writer_cnc = vcf.Writer(open(CNC, 'w'), vcf_reader)

# write all OMIM non-coding
CNC_OMIM = "/dors/capra_lab/data/clinvar/2021-07-07/GRCh38/clinvar_20210626.candidate_non-coding_OMIM.vcf"
vcf_writer_cncomim = vcf.Writer(open(CNC_OMIM, 'w'), vcf_reader)

# write all OMIM w/ no gene. Not sure non-coding status
NG_OMIM = "/dors/capra_lab/data/clinvar/2021-07-07/GRCh38/clinvar_20210626.noGENEINFO_OMIM.vcf"
vcf_writer_ngomim = vcf.Writer(open(NG_OMIM, 'w'), vcf_reader)

# parse vcf
- Find intron and UTR variants (use MC field in INFO)
    - record in candidate_non-coding.vcf
    
    
- if OMIM in intron and UTR variant annotation
    - record in candidate_non-coding_OMIM.vcf
    
    
- if OMIM in annotation AND no GENEINFO or MC 
    - record in noGENEINFO_OMIM.vcf
    
- close the files

In [3]:
val = 0

for record in vcf_reader:
    
    # Find intron and UTR variants
    if "MC" in record.INFO: 
        for i in record.INFO["MC"]:  # this is a list
            if "SO:0001627" in i:  # filter for intron_variant/UTR annotations
                vcf_writer_cnc.write_record(record)  # write to candidate non-coding region vcf
               
                if "CLNDISDB" in record.INFO:  # if OMIM annotation in CLINDISDB
                    for j in record.INFO["CLNDISDB"]:  # this is a list

                        if j is not None and "OMIM" in j:  # filter for OMIM
                            vcf_writer_cncomim.write_record(record)  # write the OMIM record to a separate file
                
    # if MC not annotated, but OMIM and no GENEINFO
    elif "CLNDISDB" in record.INFO and "GENEINFO" not in record.INFO:
        for j in record.INFO["CLNDISDB"]:  # this is a list
            if j is not None and "OMIM" in j:  # filter for OMIM
                vcf_writer_ngomim.write_record(record) # write the OMIM record to a separate file
"""
    val +=1
         
    if val > 100:
        break
"""

vcf_writer_cnc.close()
vcf_writer_cncomim.close()
vcf_writer_ngomim.close()

In [21]:
# write all OMIM non-coding
CNC_OMIM_INTRON_ONLY = "/dors/capra_lab/data/clinvar/2021-07-07/GRCh38/clinvar_20210626.candidate_INTRON_ONLY_OMIM.vcf"
vcf_writer_cncomimIO = vcf.Writer(open(CNC_OMIM_INTRON_ONLY, 'w'), vcf_reader)

In [22]:
val = 0
one_annot, multiple_annot = 0, 0

sig_dict = {}
vcf_reader = vcf.Reader(open(CNC_OMIM, 'r'))
for record in vcf_reader:
    if "CLNSIG" in record.INFO:
        for i in record.INFO["CLNSIG"]:
            sig = i
            
            if len(record.INFO["MC"])>1:
                multiple_annot +=1
            else:
                vcf_writer_cncomimIO.write_record(record) # write the intron only OMIM file
                
                one_annot +=1
                if sig not in sig_dict:
                    sig_dict[sig] = 1
                else:
                    sig_dict[sig] += 1
            if "Pathogenic" in i and len(record.INFO["MC"]) == 1:
                print(i)
                print(record, record.INFO, "\n\n")
                val+=1
                

vcf_writer_cncomimIO.close()

one_annot, multiple_annot, val

Pathogenic
Record(CHROM=1, POS=7621940, REF=CGTCTTCTCTGGGTGACCTCGTGTGACCTTGCAATCCCGTCAAGTCTTTGGTTCTGCAGAAACAACTGTCAGATCGGGAGTTAGTCACCTCCAAAGACAGAAACCAAAAACCAAAACAGTGACCTTGTTCTTTCTGTTCATGACCCTGGAGGTATGGGGGGAGGAGGAGGAAGCGGAGATTGGTTTTGCTCGTTAACTCATTATCAGTGACTGAGCGGCTACTCATGGCTGGGAACTTGTTGGGTGCCCTCCATTACACTTCCCTTAATCCTCTCAATGGCTTTTGATATCCTCAGGTAGATGGGAGGATCCCCATTTTACCTGGGAGGGATACAAGGCCAAGAGGGGTTAAGTAACTTGCCCAAGATCACACAGCGTTTGAAGTGGCCAAGGTGAAATTTGATCTAACTTCCACTTGGATGTGTGATTCCACTCCACATCCAAGTTCTTTCCATCATCTGGCTGCAGGCAAGATAACAGATTGGTGTATTCTTTCTATCTTATTTGAATTCCAAAGTTAGTATCCGCTATTGAAAACAGCTGACTTCATTATAACCCATCATTGACTCCTTTCTTTAACAAATAACCAGCAAAACAGAGCGTTCTTTCGGAGTAAGCCTTCCACCACCTGAACTTTGACAACTTTCTCCCTAATGGGTTTCTCCCTGTGAGTATTTGATCTTGGCCACCAGCATGAGCCTATCCTGGTGATCTCAAAATGCTGAGGTTTTTCTTCTGTTAAGAGATTGAATCTTCTGTTCCATGTGGAGGCTTATTCAACATTAAAGCGTTCCTGAACCATTTCATTTATTCAACAAATATTTTCCAAGCCCAGCTCTGGGCCAGATGCCAGGCTCAAGGCTTAGCAAGCCAGACAGATGTGGAGCCGCCATCATGGAACTCGGCCCTCACAGGCCATTGTCCTTGGCTTTCCAGACTGAACCTCTGTGCTTTCT

Pathogenic
Record(CHROM=1, POS=63396602, REF=G, ALT=[A]) {'ALLELEID': 20538, 'CLNDN': ['Congenital_disorder_of_glycosylation_type_1C'], 'CLNDISDB': ['MONDO:MONDO:0011291', 'MedGen:C2930997', 'OMIM:603147', 'Orphanet:ORPHA79320'], 'CLNHGVS': ['NC_000001.11:g.63396602G>A'], 'CLNREVSTAT': ['no_assertion_criteria_provided'], 'CLNSIG': ['Pathogenic'], 'CLNVC': 'single_nucleotide_variant', 'CLNVCSO': 'SO:0001483', 'CLNVI': ['OMIM_Allelic_Variant:604566.0003'], 'GENEINFO': 'ALG6:29929', 'MC': ['SO:0001627|intron_variant'], 'ORIGIN': ['1'], 'RS': ['1557585860']} 


Pathogenic/Likely_pathogenic
Record(CHROM=1, POS=63402348, REF=G, ALT=[A]) {'AF_ESP': 0.00077, 'AF_EXAC': 0.00046, 'ALLELEID': 101428, 'CLNDN': ['Congenital_disorder_of_glycosylation_type_1C|not_provided'], 'CLNDISDB': ['MONDO:MONDO:0011291', 'MedGen:C2930997', 'OMIM:603147', 'Orphanet:ORPHA79320|MedGen:CN517202'], 'CLNHGVS': ['NC_000001.11:g.63402348G>A'], 'CLNREVSTAT': ['criteria_provided', '_multiple_submitters', '_no_conflicts']

Pathogenic
Record(CHROM=1, POS=156137033, REF=G, ALT=[C]) {'ALLELEID': 77730, 'CLNDN': ['Familial_partial_lipodystrophy_2|not_provided'], 'CLNDISDB': ['MONDO:MONDO:0007906', 'MedGen:C1720860', 'OMIM:151660', 'Orphanet:ORPHA2348|MedGen:CN517202'], 'CLNHGVS': ['NC_000001.11:g.156137033G>C'], 'CLNREVSTAT': ['criteria_provided', '_single_submitter'], 'CLNSIG': ['Pathogenic'], 'CLNVC': 'single_nucleotide_variant', 'CLNVCSO': 'SO:0001483', 'CLNVI': ['OMIM_Allelic_Variant:150330.0039'], 'GENEINFO': 'LMNA:4000', 'MC': ['SO:0001627|intron_variant'], 'ORIGIN': ['1'], 'RS': ['267607543']} 


Pathogenic
Record(CHROM=1, POS=156137237, REF=G, ALT=[C]) {'ALLELEID': 77751, 'CLNDN': ['Emery-Dreifuss_muscular_dystrophy_2', '_autosomal_dominant|not_provided'], 'CLNDISDB': ['MONDO:MONDO:0021569', 'MedGen:C0410190', 'OMIM:181350', 'Orphanet:ORPHA264', 'SNOMED_CT:240072005|MedGen:CN517202'], 'CLNHGVS': ['NC_000001.11:g.156137237G>C'], 'CLNREVSTAT': ['no_assertion_criteria_provided'], 'CLNSIG': ['Pathogenic'

Pathogenic
Record(CHROM=1, POS=202334984, REF=C, ALT=[T]) {'ALLELEID': 196428, 'CLNDN': ['Fanconi_anemia', '_complementation_group_T'], 'CLNDISDB': ['MONDO:MONDO:0014638', 'MedGen:C4084840', 'OMIM:616435'], 'CLNHGVS': ['NC_000001.11:g.202334984C>T'], 'CLNREVSTAT': ['no_assertion_criteria_provided'], 'CLNSIG': ['Pathogenic'], 'CLNVC': 'single_nucleotide_variant', 'CLNVCSO': 'SO:0001483', 'CLNVI': ['OMIM_Allelic_Variant:610538.0002'], 'GENEINFO': 'UBE2T:29089', 'MC': ['SO:0001627|intron_variant'], 'ORIGIN': ['1'], 'RS': ['796052212']} 


Pathogenic
Record(CHROM=1, POS=209633136, REF=A, ALT=[G]) {'ALLELEID': 29590, 'CLNDN': ['Junctional_epidermolysis_bullosa', '_non-Herlitz_type'], 'CLNDISDB': ['MONDO:MONDO:0009180', 'MedGen:C0268374', 'OMIM:226650', 'Orphanet:ORPHA89840', 'SNOMED_CT:33662006'], 'CLNHGVS': ['NC_000001.11:g.209633136A>G'], 'CLNREVSTAT': ['no_assertion_criteria_provided'], 'CLNSIG': ['Pathogenic'], 'CLNVC': 'single_nucleotide_variant', 'CLNVCSO': 'SO:0001483', 'CLNVI': ['OM

Pathogenic
Record(CHROM=2, POS=25247759, REF=C, ALT=[T]) {'ALLELEID': 1059270, 'CLNDN': ['Tatton-Brown-rahman_syndrome'], 'CLNDISDB': ['MONDO:MONDO:0014382', 'MedGen:C4014545', 'OMIM:615879', 'Orphanet:ORPHA404443'], 'CLNHGVS': ['NC_000002.12:g.25247759C>T'], 'CLNREVSTAT': ['criteria_provided', '_single_submitter'], 'CLNSIG': ['Pathogenic'], 'CLNVC': 'single_nucleotide_variant', 'CLNVCSO': 'SO:0001483', 'GENEINFO': 'DNMT3A:1788', 'MC': ['SO:0001627|intron_variant'], 'ORIGIN': ['1']} 


Pathogenic
Record(CHROM=2, POS=26238931, REF=T, ALT=[C]) {'AF_EXAC': 9e-05, 'ALLELEID': 23770, 'CLNDN': ['Mitochondrial_trifunctional_protein_deficiency|Long-chain_3-hydroxyacyl-CoA_dehydrogenase_deficiency|not_provided'], 'CLNDISDB': ['MONDO:MONDO:0012172', 'MedGen:C1969443', 'OMIM:609015', 'Orphanet:ORPHA746', 'SNOMED_CT:237999008|MONDO:MONDO:0012173', 'MedGen:C3711645', 'OMIM:609016', 'Orphanet:ORPHA5|MedGen:CN517202'], 'CLNHGVS': ['NC_000002.12:g.26238931T>C'], 'CLNREVSTAT': ['criteria_provided', '_m

...

Pathogenic
Record(CHROM=22, POS=20996679, REF=C, ALT=[A]) {'ALLELEID': 590158, 'CLNDN': ['Noonan_syndrome_2'], 'CLNDISDB': ['MONDO:MONDO:0011531', 'MedGen:C1854469', 'OMIM:605275'], 'CLNHGVS': ['NC_000022.11:g.20996679C>A'], 'CLNREVSTAT': ['no_assertion_criteria_provided'], 'CLNSIG': ['Pathogenic'], 'CLNVC': 'single_nucleotide_variant', 'CLNVCSO': 'SO:0001483', 'CLNVI': ['OMIM_Allelic_Variant:600574.0011'], 'GENEINFO': 'LZTR1:8216', 'MC': ['SO:0001627|intron_variant'], 'ORIGIN': ['1'], 'RS': ['1249726034']} 


Pathogenic
Record(CHROM=22, POS=30615125, REF=A, ALT=[T]) {'ALLELEID': 15139, 'CLNDN': ['Transcolabamin_II_deficiency'], 'CLNDISDB': ['MONDO:MONDO:0010149', 'MedGen:C0342701', 'OMIM:275350', 'Orphanet:ORPHA859', 'SNOMED_CT:237934001'], 'CLNHGVS': ['NC_000022.11:g.30615125A>T'], 'CLNREVSTAT': ['no_assertion_criteria_provided'], 'CLNSIG': ['Pathogenic'], 'CLNVC': 'single_nucleotide_variant', 'CLNVCSO': 'SO:0001483', 'CLNVI': ['OMIM_Allelic_Variant:613441.0005'], 'GENEINFO': 'TCN2:6

Pathogenic
Record(CHROM=X, POS=13716097, REF=AATT, ALT=[A]) {'ALLELEID': 132622, 'CLNDN': ['Spondyloepiphyseal_dysplasia_tarda'], 'CLNDISDB': ['MedGen:C3541456', 'OMIM:313400', 'Orphanet:ORPHA93284', 'SNOMED_CT:51952004'], 'CLNHGVS': ['NC_000023.11:g.13716099_13716101del'], 'CLNREVSTAT': ['no_assertion_criteria_provided'], 'CLNSIG': ['Pathogenic'], 'CLNVC': 'Deletion', 'CLNVCSO': 'SO:0000159', 'CLNVI': ['OMIM_Allelic_Variant:300202.0012'], 'GENEINFO': 'TRAPPC2:6399', 'MC': ['SO:0001627|intron_variant'], 'ORIGIN': ['1'], 'RS': ['587777330']} 


Pathogenic
Record(CHROM=X, POS=13716530, REF=A, ALT=[G]) {'ALLELEID': 26555, 'CLNDN': ['Spondyloepiphyseal_dysplasia_tarda'], 'CLNDISDB': ['MedGen:C3541456', 'OMIM:313400', 'Orphanet:ORPHA93284', 'SNOMED_CT:51952004'], 'CLNHGVS': ['NC_000023.11:g.13716530A>G'], 'CLNREVSTAT': ['no_assertion_criteria_provided'], 'CLNSIG': ['Pathogenic'], 'CLNVC': 'single_nucleotide_variant', 'CLNVCSO': 'SO:0001483', 'CLNVI': ['OMIM_Allelic_Variant:300202.0010'], 'G

Pathogenic
Record(CHROM=X, POS=78011257, REF=G, ALT=[A]) {'ALLELEID': 209231, 'CLNDN': ['Menkes_kinky-hair_syndrome'], 'CLNDISDB': ['MONDO:MONDO:0010651', 'MedGen:C0022716', 'OMIM:309400', 'Orphanet:ORPHA565', 'SNOMED_CT:59178007'], 'CLNHGVS': ['NC_000023.11:g.78011257G>A'], 'CLNREVSTAT': ['criteria_provided', '_single_submitter'], 'CLNSIG': ['Pathogenic'], 'CLNVC': 'single_nucleotide_variant', 'CLNVCSO': 'SO:0001483', 'GENEINFO': 'ATP7A:538', 'MC': ['SO:0001627|intron_variant'], 'ORIGIN': ['1'], 'RS': ['797045341']} 


Pathogenic/Likely_pathogenic
Record(CHROM=X, POS=78011679, REF=G, ALT=[C]) {'ALLELEID': 209241, 'CLNDN': ['Distal_spinal_muscular_atrophy', '_X-linked_3|Cutis_laxa', '_X-linked|Menkes_kinky-hair_syndrome'], 'CLNDISDB': ['MONDO:MONDO:0010338', 'MedGen:C1845359', 'OMIM:300489', 'Orphanet:ORPHA139557|MONDO:MONDO:0010572', 'MedGen:C0268353', 'OMIM:304150', 'Orphanet:ORPHA198', 'SNOMED_CT:59399004|MONDO:MONDO:0010651', 'MedGen:C0022716', 'OMIM:309400', 'Orphanet:ORPHA565', '

Pathogenic
Record(CHROM=X, POS=154363633, REF=A, ALT=[T]) {'ALLELEID': 481079, 'CLNDN': ['Intestinal_pseudoobstruction', '_neuronal', '_chronic_idiopathic', '_X-linked|Periventricular_nodular_heterotopia_1'], 'CLNDISDB': ['MONDO:MONDO:0010232', 'MedGen:C2746068', 'OMIM:300048|MONDO:MONDO:0010233', 'MedGen:C1848213', 'OMIM:300049', 'SNOMED_CT:448227009'], 'CLNHGVS': ['NC_000023.11:g.154363633A>T'], 'CLNREVSTAT': ['no_assertion_criteria_provided'], 'CLNSIG': ['Pathogenic'], 'CLNVC': 'single_nucleotide_variant', 'CLNVCSO': 'SO:0001483', 'GENEINFO': 'FLNA:2316', 'MC': ['SO:0001627|intron_variant'], 'ORIGIN': ['32'], 'RS': ['1557178535']} 


Pathogenic
Record(CHROM=X, POS=154363633, REF=A, ALT=[T]) {'ALLELEID': 481079, 'CLNDN': ['Intestinal_pseudoobstruction', '_neuronal', '_chronic_idiopathic', '_X-linked|Periventricular_nodular_heterotopia_1'], 'CLNDISDB': ['MONDO:MONDO:0010232', 'MedGen:C2746068', 'OMIM:300048|MONDO:MONDO:0010233', 'MedGen:C1848213', 'OMIM:300049', 'SNOMED_CT:448227009']

(63354, 64432, 1549)

In [12]:
one_annot, multiple_annot, val

(63354, 64432, 1549)

In [14]:
sig_dict

{'Uncertain_significance': 16527,
 'Likely_benign': 23559,
 'Benign': 8851,
 'Benign/Likely_benign': 3505,
 'Conflicting_interpretations_of_pathogenicity': 7767,
 'Pathogenic': 1265,
 'Likely_pathogenic': 500,
 'association': 45,
 'Pathogenic/Likely_pathogenic': 284,
 'risk_factor': 20,
 'not_provided': 982,
 'Affects': 5,
 'drug_response': 5,
 '_drug_response': 5,
 '_other': 7,
 'other': 19,
 '_association': 2,
 '_protective': 2,
 '_risk_factor': 3,
 'protective': 1}

In [17]:
val = 0
one_annot, multiple_annot = 0, 0

sig_dict = {}
vcf_reader = vcf.Reader(open(NG_OMIM, 'r'))
for record in vcf_reader:
    if "CLNSIG" in record.INFO:
        for i in record.INFO["CLNSIG"]:
            sig = i
            

            if sig not in sig_dict:
                sig_dict[sig] = 1
            else:
                sig_dict[sig] += 1
            if "Pathogenic" in i:
                print(i)
                print(record, record.INFO, "\n\n")
                val+=1
                
    one_annot+=1
    

one_annot, multiple_annot, val

Pathogenic
Record(CHROM=4, POS=10072573, REF=G, ALT=[A]) {'ALLELEID': 798969, 'CLNDN': ['Intellectual_developmental_disorder_61'], 'CLNDISDB': ['MONDO:MONDO:0032485', 'MedGen:C5231400', 'OMIM:618009'], 'CLNHGVS': ['NC_000004.12:g.10072573G>A'], 'CLNREVSTAT': ['no_assertion_criteria_provided'], 'CLNSIG': ['Pathogenic'], 'CLNVC': 'single_nucleotide_variant', 'CLNVCSO': 'SO:0001483', 'CLNVI': ['OMIM_Allelic_Variant:603808.0004'], 'ORIGIN': ['1'], 'RS': ['540561511']} 


Pathogenic
Record(CHROM=7, POS=62535490, REF=A, ALT=[G]) {'ALLELEID': 94317, 'CLNDN': ['Isolated_growth_hormone_deficiency_type_IB'], 'CLNDISDB': ['MONDO:MONDO:0013006', 'MedGen:C2748571', 'OMIM:612781', 'Orphanet:ORPHA231671'], 'CLNHGVS': ['NC_000007.14:g.62535490A>G'], 'CLNREVSTAT': ['no_assertion_criteria_provided'], 'CLNSIG': ['Pathogenic'], 'CLNVC': 'single_nucleotide_variant', 'CLNVCSO': 'SO:0001483', 'ORIGIN': ['4'], 'RS': ['483352872']} 


Pathogenic
Record(CHROM=8, POS=228116, REF=G, ALT=[C]) {'ALLELEID': 624149, 

(41, 0, 15)

In [18]:
one_annot, multiple_annot, val

(41, 0, 15)